In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import os
from vzam.searcher import FaissRhashVideoSearcher
from sklearn.metrics import classification_report
%matplotlib inline

In [3]:
initial_cwd = os.getcwd()
initial_cwd

'/home/boris/study/курсовой/vzam'

In [4]:
# print(initial_cwd)
# os.chdir(initial_cwd)
# os.chdir('..')
# print(os.getcwd())

In [5]:
from vzam import *

In [46]:
vcdb = pd.read_csv('data/vcdb.csv')
vcdb_lut = pd.read_csv('data/vcdb_query_source_lut.csv')
vcdb_lut.index = vcdb_lut['query_id']

In [47]:
vcdb.head()

,query,source,source_id
0,core_dataset/baggio_penalty_1994\3504e360accba...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
1,core_dataset/baggio_penalty_1994\37b31d607d31a...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
2,core_dataset/baggio_penalty_1994\458db5aa227ae...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
3,core_dataset/baggio_penalty_1994\5c5714c0a56fd...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv
4,core_dataset/baggio_penalty_1994\67824b87c0698...,core_dataset/baggio_penalty_1994\bb604f57a1845...,bb604f57a18455867544e79c2e32bf5583c358d4.flv


In [48]:
vcdb_lut.head()

,query_id,source_id
query_id,,
3504e360accbaccb1580befbb441f1019664c2bb.mp4,3504e360accbaccb1580befbb441f1019664c2bb.mp4,bb604f57a18455867544e79c2e32bf5583c358d4.flv
37b31d607d31a47d347b15dae2b8aa63e57861eb.flv,37b31d607d31a47d347b15dae2b8aa63e57861eb.flv,bb604f57a18455867544e79c2e32bf5583c358d4.flv
458db5aa227ae49ceb8bc1bed5f9cf5b4bed63f2.flv,458db5aa227ae49ceb8bc1bed5f9cf5b4bed63f2.flv,bb604f57a18455867544e79c2e32bf5583c358d4.flv
5c5714c0a56fd2a96f99db2f59b0d03659d77cdf.flv,5c5714c0a56fd2a96f99db2f59b0d03659d77cdf.flv,bb604f57a18455867544e79c2e32bf5583c358d4.flv
67824b87c0698c499acad123b7498ae17f97bf6d.flv,67824b87c0698c499acad123b7498ae17f97bf6d.flv,bb604f57a18455867544e79c2e32bf5583c358d4.flv


In [50]:
vcdb_lut.shape

(106, 2)

In [51]:
# vcdb['query_id'] = vcdb['query'].apply(lambda x: os.path.basename(x).split('\\')[-1])
# vcdb.head()

In [52]:
# vcdb_lut = vcdb[['query_id', 'source_id']]
# vcdb_lut.index = vcdb_lut.query_id

In [53]:

vcdb_lut.ix['3504e360accbaccb1580befbb441f1019664c2bb.mp4'].source_id

'bb604f57a18455867544e79c2e32bf5583c358d4.flv'

In [54]:
SOURCE_DATASET = 'data/vcdb_rhash_source.csv'
QUERY_DATASET = 'data/vcdb_rhash_query.csv'

In [55]:
def list_converter(text):
    return eval(text.replace(' ', ','))

source_df = pd.read_csv(SOURCE_DATASET, converters={"feature": list_converter})
source_df.head()

,feature,ts,id
0,"[0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, ...",0.45,bb604f57a18455867544e79c2e32bf5583c358d4.flv
1,"[0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",0.95,bb604f57a18455867544e79c2e32bf5583c358d4.flv
2,"[0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",1.45,bb604f57a18455867544e79c2e32bf5583c358d4.flv
3,"[0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, ...",1.95,bb604f57a18455867544e79c2e32bf5583c358d4.flv
4,"[0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, ...",2.45,bb604f57a18455867544e79c2e32bf5583c358d4.flv


In [56]:
source_df.shape

(2677, 3)

In [57]:
frame_times = source_df['ts']
labels = source_df.id.values
frame_vectors = np.vstack(source_df['feature'].values)
frame_vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0]])

In [58]:
searcher = FaissRhashVideoSearcher(frame_vectors, labels, frame_times)


In [59]:
query_df = pd.read_csv(QUERY_DATASET, converters={"feature": list_converter})
query_df.head()

,feature,ts,id
0,"[1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, ...",0.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4
1,"[0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",1.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4
2,"[0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",2.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4
3,"[0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",3.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4
4,"[1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",4.8,3504e360accbaccb1580befbb441f1019664c2bb.mp4


In [60]:
query_df.shape

(94619, 3)

In [61]:
query_ids = query_df.id.unique()
len(query_ids)

519

In [62]:
searcher.dist_threshold = 20

In [67]:
import time

In [68]:
true_labels = []
predictions = []
times = []
for i, query_id in enumerate(query_ids[:5]):
    start_time = time.time()
    tdf = query_df[query_df.id == query_id]
    try:
        true_label = vcdb_lut.ix[query_id].source_id
    except KeyError:
        true_label = 'miss'
    true_labels.append('miss' if true_label != true_label else true_label)
    query_vectors = np.vstack(tdf['feature'].values)
    prediction = searcher.lookup(query_vectors, conf_threshold=0.7)
    predicted_label = prediction[0][0] if prediction else 'miss'
    predictions.append(predicted_label)
    times.append(time.time() - start_time)
    if i % 50 == 0:
        print(i)
print(classification_report(true_labels, predictions))

0
                                              precision    recall  f1-score   support

bb604f57a18455867544e79c2e32bf5583c358d4.flv       0.00      0.00      0.00         5
                                        miss       0.00      0.00      0.00         0

                                   micro avg       0.00      0.00      0.00         5
                                   macro avg       0.00      0.00      0.00         5
                                weighted avg       0.00      0.00      0.00         5



In [69]:
print(np.mean(times))

0.15899910926818847


In [22]:
# predictions

In [70]:
true_labels = []
predictions = []
times = []
for i, query_id in enumerate(query_ids):
    start_time = time.time()
    tdf = query_df[query_df.id == query_id]
    try:
        true_label = vcdb_lut.ix[query_id].source_id
    except KeyError:
        true_label = 'miss'
    true_labels.append('miss' if true_label is np.NaN else true_label)
    query_vectors = np.vstack(tdf['feature'].values)
    prediction = searcher.lookup_sequence(query_vectors)
    prediction = prediction[0] if prediction else None
    predicted_label = prediction['label'] if prediction else 'miss'
    predictions.append(predicted_label)
    times.append(time.time() - start_time)
    if i % 50 == 0:
        print(i)
    
print(classification_report(true_labels, predictions))
print(np.mean(times))

0
50
100
150
200
250
300
350
400
450
500
                                              precision    recall  f1-score   support

098df9c477ecf6c4f3818b198490b0258163015d.flv       0.17      0.09      0.12        11
127dab55025984673f65d3a23b1fea99ecc79b15.mp4       0.11      0.27      0.16        26
3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv       0.13      0.18      0.15        11
3df02ec5ddb74647d8fb96f08adba8e152050ef9.flv       0.70      1.00      0.82         7
410ba1c253fb8d4a8ba6e59aa9e39d39cb3b245b.flv       1.00      0.33      0.50         6
46f2e964ae16f5c27fad70d6849c76616fad7502.flv       0.00      0.00      0.00         4
97c71f57e83f6ab16906fdd7edc8e3f88d2ceee1.flv       0.07      0.12      0.09        24
bb604f57a18455867544e79c2e32bf5583c358d4.flv       0.00      0.00      0.00        10
d2015b438b70f022967713d6f977ebc67a16839e.flv       0.00      0.00      0.00         7
                                        miss       0.85      0.73      0.79       413

           

In [71]:
# Majority vote
true_labels = []
predictions = []
times = []
for i, query_id in enumerate(query_ids):
    start_time = time.time()
    tdf = query_df[query_df.id == query_id]
    try:
        true_label = vcdb_lut.ix[query_id].source_id
    except KeyError:
        true_label = 'miss'
    true_labels.append('miss' if true_label is np.NaN else true_label)
    query_vectors = np.vstack(tdf['feature'].values)
    prediction = searcher.lookup_mv(query_vectors)
    predicted_label = prediction[0] if prediction else 'miss'
    predictions.append(predicted_label)
    times.append(time.time() - start_time)
    if i % 50 == 0:
        print(i)
print(classification_report(true_labels, predictions))
print(np.mean(times))

0
50
100
150
200
250
300
350
400
450
500
                                              precision    recall  f1-score   support

098df9c477ecf6c4f3818b198490b0258163015d.flv       0.67      0.18      0.29        11
127dab55025984673f65d3a23b1fea99ecc79b15.mp4       0.46      0.81      0.58        26
3499b9fd4316131ebf0f5d9bbecf6f3f8600c4fb.flv       1.00      0.18      0.31        11
3df02ec5ddb74647d8fb96f08adba8e152050ef9.flv       0.44      1.00      0.61         7
410ba1c253fb8d4a8ba6e59aa9e39d39cb3b245b.flv       1.00      0.67      0.80         6
46f2e964ae16f5c27fad70d6849c76616fad7502.flv       0.00      0.00      0.00         4
97c71f57e83f6ab16906fdd7edc8e3f88d2ceee1.flv       0.05      0.17      0.07        24
bb604f57a18455867544e79c2e32bf5583c358d4.flv       0.00      0.00      0.00        10
d2015b438b70f022967713d6f977ebc67a16839e.flv       0.00      0.00      0.00         7
                                        miss       0.87      0.76      0.81       413

           

In [45]:
import scipy

def ham_dist(vecs1, vecs2):
    dists = []
    for u, v in zip(vecs1, vecs2):
        dists.append(scipy.spatial.distance.hamming(u, v))
    return dists

def sequential_search(searcher, vectors):
    # df = pd.DataFrame({'vectors': searcher.vectors, 'labels': searcher.labels}, index=range(len(searcher.vectors)))
    min_dist = None
    min_label = None
    
    for label in set(searcher.labels):
        mask = searcher.labels == label
        label_vectors = searcher.vectors[mask]
        
        for i in range(len(label_vectors) - len(vectors)):
            vecs = label_vectors[i:i+len(vectors)]
            
            dists = ham_dist(vectors, vecs)
            
            if min_dist is None or sum(dists) < min_dist:
                min_dist = sum(dists)
                min_label = label
                
    return min_label, min_dist

true_labels = []
predictions = []
for i, query_id in enumerate(query_ids):
    tdf = query_df[query_df.id == query_id]
    try:
        true_label = vcdb_lut.ix[query_id].source_id
    except KeyError:
        true_label = 'miss'
    true_labels.append('miss' if true_label is np.NaN else true_label)
    query_vectors = np.vstack(tdf['feature'].values)
    prediction = sequential_search(searcher, query_vectors)
    predicted_label = prediction[0] if prediction else 'miss'
    predictions.append(predicted_label)
    
    if i % 50 == 0:
        print(i)
print(classification_report(true_labels, predictions))

0
50
100
150
200
250


KeyboardInterrupt: 

In [39]:
sequential_search(searcher, query_vectors)

(2.2600000000000002, 'd2015b438b70f022967713d6f977ebc67a16839e.flv')